# EDA Basic Lesson Plan

Welcome to the Exploratory Data Analysis (EDA) lesson! This notebook is designed to guide you through the essential steps of taking a raw dataset, understanding its structure, cleaning it, and transforming it for analysis.

**Structure:**
* **Section 1: The First Look:** How to read files and get a quick statistical summary.
* **Section 2: The Cleanup:** Handling the messy reality of data (missing values, duplicates, outliers).
* **Section 3: Refinement & Transformation:** preparing data for machine learning or deep analysis (mapping, string manipulation).

**For Instructors:** Use the narrative to introduce concepts before running code cells.
**For Learners:** Read the comments in the code to understand exactly what each parameter does.

### Setup
As always, we start by importing the necessary libraries. `pandas` is our primary tool for data manipulation, and `numpy` helps with numerical operations.

In [199]:
import pandas as pd
import numpy as np

---

# Section 1: The First Look: Inspection & Summary

**Goal:** Load data from different file formats and get a "health check" of the data using descriptive statistics.

We will cover:
1. Reading CSV and Text files (handling headers, delimiters, and special characters).
2. Summarizing data using `.describe()`, `.sum()`, and `.value_counts()`.

## Part 1: Reading Data

Pandas is incredibly flexible with input formats. Let's start by reading a standard Comma-Separated Values (CSV) file.

In [200]:
# Let's inspect the raw file content first using a shell command
!cat ../data/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [201]:
# Use read_csv to load the data into a DataFrame
# By default, it assumes the first row is the header
df = pd.read_csv("../data/ex1.csv")

df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


**Scenario:** What if the file doesn't have a header row? If we don't specify this, pandas will mistakenly use the first row of data as column names.

In [202]:
!cat ../data/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [203]:
# Option 1: Tell pandas there is no header. It will assign integers (0, 1, 2...) as column names.
pd.read_csv("../data/ex2.csv", header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [204]:
# Option 2: Provide your own column names using the 'names' parameter
pd.read_csv("../data/ex2.csv", names=["a", "b", "c", "d", "message"])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


**Indexing:** You can also designate a specific column to be the index (row labels) of the DataFrame, rather than the default 0, 1, 2... index.

In [205]:
names = ["a", "b", "c", "d", "message"]

# Use 'index_col' to set the 'message' column as the index
pd.read_csv("../data/ex2.csv", names=names, index_col="message")

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


**Irregular Separators:** Sometimes data isn't separated by commas. It might be tabs, spaces, or a variable amount of whitespace. We can use Regular Expressions (Regex) to handle this.

In [206]:
# Inspect a file with messy whitespace
!cat ../data/ex3.txt

A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491

In [207]:
# sep="\s+" is a regex that means "one or more whitespace characters"
result = pd.read_csv("../data/ex3.txt", sep="\s+")

result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


**Skipping Rows:** Sometimes files contain comments or metadata at the top that we want to ignore.

In [208]:
!cat ../data/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [209]:
# Skip specific rows by index (0, 2, and 3 here) to get to the clean data
pd.read_csv("../data/ex4.csv", skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


**Handling Missing Values (at load time):** Pandas is smart about identifying missing data (empty strings, 'NA', 'NULL'), but we can also define our own "sentinels" for missing values.

In [210]:
!cat ../data/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,NULL

In [211]:
# Default behavior: reads 'NA' and 'NULL' as NaN (Not a Number/Missing)
result = pd.read_csv("../data/ex5.csv")

result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


In [212]:
# Verify where the missing values are detected
result.isna()

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,True


You can customize what is considered "Missing" for each column specifically.

In [213]:
# Define a dictionary of sentinels
# For column 'message', treat 'foo' and 'NA' as null
# For column 'something', treat 'two' as null
sentinels = {"message": ["NULL", "NA"], "something": ["two"]}

pd.read_csv("../data/ex5.csv", na_values=sentinels, keep_default_na=False)

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,NaN,5,6,,8,world
2,three,9,10,11,12,NaN


**Reading Excel:** Reading Excel files works similarly, but we can specify sheet names.

In [214]:
# Load the Excel file object
xlsx = pd.ExcelFile("../data/Resaleflatpricesbasedonregistrationdate.xlsx")

# List available sheets
xlsx.sheet_names

['2017']

In [215]:
# Parse a specific sheet into a DataFrame
xlsx.parse(sheet_name="2017")

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44,Improved,1979,61 years 04 months,232000
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67,New Generation,1978,60 years 07 months,250000
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67,New Generation,1980,62 years 05 months,262000
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68,New Generation,1980,62 years 01 month,265000
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67,New Generation,1980,62 years 05 months,265000
...,...,...,...,...,...,...,...,...,...,...,...
1180,2017-01,YISHUN,EXECUTIVE,387,YISHUN RING RD,07 TO 09,142,Apartment,1988,70 years 05 months,587000
1181,2017-01,YISHUN,EXECUTIVE,877,YISHUN ST 81,04 TO 06,142,Apartment,1987,69 years 11 months,590000
1182,2017-01,YISHUN,EXECUTIVE,836,YISHUN ST 81,04 TO 06,146,Maisonette,1988,70 years 01 month,673000
1183,2017-01,YISHUN,EXECUTIVE,633A,YISHUN ST 61,01 TO 03,164,Apartment,1992,74 years 05 months,678000


In [216]:
# Shortcut: Read directly without creating an ExcelFile object first
frame = pd.read_excel("../data/Resaleflatpricesbasedonregistrationdate.xlsx", sheet_name="2017")

frame

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44,Improved,1979,61 years 04 months,232000
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67,New Generation,1978,60 years 07 months,250000
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67,New Generation,1980,62 years 05 months,262000
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68,New Generation,1980,62 years 01 month,265000
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67,New Generation,1980,62 years 05 months,265000
...,...,...,...,...,...,...,...,...,...,...,...
1180,2017-01,YISHUN,EXECUTIVE,387,YISHUN RING RD,07 TO 09,142,Apartment,1988,70 years 05 months,587000
1181,2017-01,YISHUN,EXECUTIVE,877,YISHUN ST 81,04 TO 06,142,Apartment,1987,69 years 11 months,590000
1182,2017-01,YISHUN,EXECUTIVE,836,YISHUN ST 81,04 TO 06,146,Maisonette,1988,70 years 01 month,673000
1183,2017-01,YISHUN,EXECUTIVE,633A,YISHUN ST 61,01 TO 03,164,Apartment,1992,74 years 05 months,678000


## Part 2: Summarizing and Computing Descriptive Statistics

Once data is loaded, we need to understand its "shape". We use summary statistics to get a birds-eye view.

> **📊 Visual Illustration Available**: See ![descriptive_statistics](../visualisations/01_descriptive_statistics.png) for visual representations of descriptive statistics including histograms, box plots, and how statistics like mean, median, and quartiles relate to data distributions.


Let's create a sample DataFrame with some missing values to test our summary methods.

In [217]:
df = pd.DataFrame([[1.4, np.nan], [7.1, -4.5], [np.nan, np.nan], [0.75, -1.3]],
                  index=["a", "b", "c", "d"], columns=["one", "two"])

df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


**Reductions:** Methods like `.sum()` or `.mean()` reduce a Series of numbers to a single value. By default, they operate down the rows (`axis=0`).

In [218]:
# Sums down the rows (returns sum for each column)
df.sum()

one    9.25
two   -5.80
dtype: float64

In [219]:
# Sums across the columns (returns sum for each row)
# axis=1 is synonymous with axis='columns'
df.sum(axis="columns")

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

**Handling NAs in Calculations:** By default, pandas ignores NaNs (treats them as zero for sums). We can change this with `skipna`.

In [220]:
# If any value is NaN, the result is NaN
df.sum(skipna=False)

one   NaN
two   NaN
dtype: float64

In [221]:
df.sum(axis=1, skipna=False)

a     NaN
b    2.60
c     NaN
d   -0.55
dtype: float64

**Indirect Statistics:** Finding *where* the max or min value is located (the index label).

In [222]:
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


In [223]:
# Returns the index of the maximum value
df.idxmax()

one    b
two    d
dtype: object

In [224]:
# Returns the index of the minimum value
df.idxmin()

one    d
two    b
dtype: object

**Accumulations:** Computing cumulative sums.

In [225]:
df.cumsum()

,one,two
a,1.40,NaN
b,8.50,-4.5
c,NaN,NaN
d,9.25,-5.8


**The `.describe()` method:** This is your best friend for a quick snapshot of numeric data.

In [226]:
# Provides count, mean, std, min, quartiles, and max
df.describe()

,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,1.075000,-3.700000
50%,1.400000,-2.900000
75%,4.250000,-2.100000
max,7.100000,-1.300000


**Categorical/Non-Numeric Data:** `.describe()` behaves differently for string data, showing counts and uniqueness.

In [227]:
obj = pd.Series(["c", "a", "d", "a", "b", "b", "c", "c"])

# For object data, we get count, unique, top (most frequent), and freq
obj.describe()

count     8
unique    4
top       c
freq      3
dtype: object

In [228]:
# Get unique values
obj.unique()

array(['c', 'a', 'd', 'b'], dtype=object)

In [229]:
# Get frequency counts of each unique value
obj.value_counts()

c    3
a    2
b    2
d    1
dtype: int64

**Exercise:** Sort value counts in ascending order.

In [230]:
obj.value_counts().sort_index()

a    2
b    2
c    3
d    1
dtype: int64

---

# Section 2: The Cleanup: Missing Data, Duplicates & Outliers

**Goal:** Real-world data is dirty. In this section, we will learn techniques to identify and fix common quality issues.

We will cover:
1. **Missing Data:** Detecting nulls and deciding whether to drop them or fill them.
2. **Duplicates:** Finding and removing repeated records.
3. **Outliers:** identifying extreme values using boolean indexing.

## Part 1: Handling Missing Data

Missing data is often represented as `NaN` (Not a Number) or `None`.

> **📊 Visual Illustration Available**: See ![missing_data](../visualisations/03_missing_data.png) for visualizations of missing data patterns, heatmaps, and comparison of different strategies for handling missing values.


In [231]:
float_data = pd.Series([1.2, -3.5, np.nan, 0])

float_data

0    1.2
1   -3.5
2    NaN
3    0.0
dtype: float64

In [232]:
# .isna() returns a boolean mask (True if missing, False if present)
float_data.isna()

0    False
1    False
2     True
3    False
dtype: bool

The built-in Python `None` value is also treated as NA in pandas object arrays.

In [233]:
string_data = pd.Series(["aardvark", np.nan, None, "avocado"])

string_data

0    aardvark
1         NaN
2        None
3     avocado
dtype: object

In [234]:
string_data.isna()

0    False
1     True
2     True
3    False
dtype: bool

### Strategy 1: Dropping Missing Data (`dropna`)
The simplest strategy is to just remove the rows or columns that contain missing values.

In [235]:
data = pd.Series([1, np.nan, 3.5, np.nan, 7])
print(data)
# Removes all NaN values from the Series
data.dropna()

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64


0    1.0
2    3.5
4    7.0
dtype: float64

In [236]:
# This is equivalent to boolean filtering:
data[data.notna()]

0    1.0
2    3.5
4    7.0
dtype: float64

With DataFrames, `dropna` by default drops **any row** containing **any missing value**.

In [237]:
data = pd.DataFrame([[1., 6.5, 3.], [1., np.nan, np.nan], 
                     [np.nan, np.nan, np.nan], [np.nan, 6.5, 3.]])

data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [238]:
# Drops rows 1, 2, and 3 because they have at least one NaN
data.dropna()

,0,1,2
0,1.0,6.5,3.0


We can control this behavior. `how='all'` only drops rows where **all** values are NaN.

In [239]:
data.dropna(how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


To drop **columns** instead of rows, pass `axis=1`.

In [240]:
# Let's add a column of all NaNs first
data[4] = np.nan

data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [241]:
# Drops the column '4' because it is all NaNs
data.dropna(axis="columns", how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


We can also set a **threshold**: keep only rows containing at least `n` valid observations.

In [242]:
df = pd.DataFrame(np.random.standard_normal((7, 3)))
# Set some missing values
df.iloc[:4, 1] = np.nan
df.iloc[:2, 2] = np.nan

df

,0,1,2
0,-0.559486,NaN,NaN
1,-0.428720,NaN,NaN
2,0.653840,NaN,-0.287815
3,3.310841,NaN,-0.112204
4,0.366587,0.707428,-0.426665
5,0.712203,0.144710,0.947274
6,-0.060332,-0.334697,-0.140714


In [243]:
df.dropna()

,0,1,2
4,0.366587,0.707428,-0.426665
5,0.712203,0.144710,0.947274
6,-0.060332,-0.334697,-0.140714


In [244]:
# Keep rows that have at least 2 non-NaN values
df.dropna(thresh=2)

,0,1,2
2,0.653840,NaN,-0.287815
3,3.310841,NaN,-0.112204
4,0.366587,0.707428,-0.426665
5,0.712203,0.144710,0.947274
6,-0.060332,-0.334697,-0.140714


### Strategy 2: Filling Missing Data (`fillna`)
Instead of losing data, we can fill the holes with a constant or a calculated value.

In [245]:
# Replace all NaNs with 0
df.fillna(0)

,0,1,2
0,-0.559486,0.000000,0.000000
1,-0.428720,0.000000,0.000000
2,0.653840,0.000000,-0.287815
3,3.310841,0.000000,-0.112204
4,0.366587,0.707428,-0.426665
5,0.712203,0.144710,0.947274
6,-0.060332,-0.334697,-0.140714


You can specify different fill values for each column:

In [246]:
# Fill column 1 with 0.5, and column 2 with 0
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-0.559486,0.500000,0.000000
1,-0.428720,0.500000,0.000000
2,0.653840,0.500000,-0.287815
3,3.310841,0.500000,-0.112204
4,0.366587,0.707428,-0.426665
5,0.712203,0.144710,0.947274
6,-0.060332,-0.334697,-0.140714


**Forward/Backward Fill:** Useful for time-series data, where you propagate the last valid observation forward or backward.

In [247]:
# Propagate next valid value backward to fill gaps
df.fillna(method="bfill")

,0,1,2
0,-0.559486,0.707428,-0.287815
1,-0.428720,0.707428,-0.287815
2,0.653840,0.707428,-0.287815
3,3.310841,0.707428,-0.112204
4,0.366587,0.707428,-0.426665
5,0.712203,0.144710,0.947274
6,-0.060332,-0.334697,-0.140714


In [248]:
# Same, but limit how many rows are filled consecutively
df.fillna(method="bfill", limit=2)

,0,1,2
0,-0.559486,NaN,-0.287815
1,-0.428720,NaN,-0.287815
2,0.653840,0.707428,-0.287815
3,3.310841,0.707428,-0.112204
4,0.366587,0.707428,-0.426665
5,0.712203,0.144710,0.947274
6,-0.060332,-0.334697,-0.140714


**Imputation:** Filling with the mean or median is a very common technique.

In [249]:
data = pd.Series([1., np.nan, 3.5, np.nan, 7])

data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [250]:
# Fill with the mean of the available data
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

### Exercise: Handling Missing Data
Practice using these methods on the dataframe below.

In [251]:
df = pd.DataFrame(np.random.standard_normal((6, 3)))

df.iloc[[2,4,5], 1] = np.nan
df.iloc[4:, 2] = np.nan
df.iloc[3:5, 0] = np.nan

df

,0,1,2
0,0.171912,0.010617,-1.143201
1,0.300648,0.282673,1.296494
2,-0.679568,NaN,-0.795177
3,NaN,-1.789043,-1.913776
4,NaN,NaN,NaN
5,-0.820984,NaN,NaN


> **Tasks:**
> 1. Remove rows with *any* missing values.
> 2. Remove rows with *all* missing values.
> 3. Fill missing values with forward fill.
> 4. Fill missing values with mean of the column.

In [252]:
print("remove rows with any missing values", df.dropna(axis="index"))
print("\nremove rows with all missing values", df.dropna(axis="index", how="all"))
print("\nfill missing values with forawrd fill", df.fillna(method="ffill"))
print("\nmean of df:\n", df.mean())
print("\ndf:\n", df)
print("\nfill missing values with mean", df.fillna(df.mean()))

remove rows with any missing values           0         1         2
0  0.171912  0.010617 -1.143201
1  0.300648  0.282673  1.296494

remove rows with all missing values           0         1         2
0  0.171912  0.010617 -1.143201
1  0.300648  0.282673  1.296494
2 -0.679568       NaN -0.795177
3       NaN -1.789043 -1.913776
5 -0.820984       NaN       NaN

fill missing values with forawrd fill           0         1         2
0  0.171912  0.010617 -1.143201
1  0.300648  0.282673  1.296494
2 -0.679568  0.282673 -0.795177
3 -0.679568 -1.789043 -1.913776
4 -0.679568 -1.789043 -1.913776
5 -0.820984 -1.789043 -1.913776

mean of df:
 0   -0.256998
1   -0.498584
2   -0.638915
dtype: float64

df:
           0         1         2
0  0.171912  0.010617 -1.143201
1  0.300648  0.282673  1.296494
2 -0.679568       NaN -0.795177
3       NaN -1.789043 -1.913776
4       NaN       NaN       NaN
5 -0.820984       NaN       NaN

fill missing values with mean           0         1         2
0  0.171912 

## Part 2: Handling Duplicates

Duplicate rows can skew analysis and models. We typically identify them and remove them.

> **📊 Visual Illustration Available**: See ![duplicates](../visualisations/06_duplicates.png) for visualizations showing how duplicates affect datasets and the impact of different `keep` parameter options.


In [253]:
data = pd.DataFrame({"k1": ["one", "two"] * 3 + ["two"],
                     "k2": [1, 1, 2, 3, 3, 4, 4]})

data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


`duplicated()` returns a boolean Series indicating whether each row has been seen before.

In [254]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

`drop_duplicates()` creates a new DataFrame with the duplicates removed.

In [255]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


**Subset:** Sometimes we only care about duplicates in specific columns.

In [256]:
data["v1"] = range(7)

data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [257]:
# Drop duplicates considering only column 'k1'
data.drop_duplicates(subset=["k1"])

,k1,k2,v1
0,one,1,0
1,two,1,1


**Keep:** By default, it keeps the first occurrence. We can keep the last one instead.

In [258]:
data.drop_duplicates(subset=["k1", "k2"], keep="last")

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


## Part 3: Handling Outliers

Outliers are extreme values that deviate significantly from the rest of the data. We can filter them using boolean indexing.

> **📊 Visual Illustration Available**: See ![outliers](../visualisations/04_outliers.png) for visualizations of outlier detection methods (IQR, Z-score) and comparison of different handling strategies.


In [259]:
# Create a dataset with normal distribution
data = pd.DataFrame(np.random.standard_normal((1000, 4)))

data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.003885,-0.060788,-0.001718,-0.032896
std,0.995680,0.986831,1.025535,0.989442
min,-3.581092,-3.247428,-4.010215,-3.793438
25%,-0.610949,-0.690710,-0.633737,-0.697984
50%,0.002066,-0.058677,-0.015031,-0.062742
75%,0.674379,0.582628,0.677254,0.605943
max,4.075599,3.224841,3.372621,3.184947


**Detection:** Let's find values exceeding 3 in absolute value (Standard Deviation > 3).

In [260]:
col = data[2]

# Boolean indexing to find rows where absolute value > 3
col[col.abs() > 3]

71     3.372621
72     3.111513
75    -4.010215
376   -3.142637
403   -3.226964
498    3.149928
741   -3.039147
Name: 2, dtype: float64

In [261]:
data.head()

,0,1,2,3
0,0.325437,0.044449,-0.494591,1.279838
1,-1.062924,-0.475014,-0.562591,0.041950
2,1.132614,-1.285826,0.356707,-1.237236
3,0.579027,-0.158589,1.369788,-0.673912
4,0.047617,1.619236,1.624633,0.185832


To find **any row** that has an outlier in **any column**, we use `.any(axis=1)`.

In [262]:
# 1. data.abs() > 3 returns a boolean DataFrame
# 2. .any(axis="columns") checks if any value in the row is True
data[(data.abs() > 3).any(axis="columns")]

,0,1,2,3
40,-3.536420,0.647258,-0.485949,-0.756612
71,-0.396388,0.806610,3.372621,-0.460297
72,-0.906772,-1.964442,3.111513,1.453419
75,1.612793,-1.720448,-4.010215,1.798855
250,-3.085033,0.411530,-0.075928,0.091853
260,-3.581092,1.242264,-0.244821,-1.301825
347,4.075599,-0.324148,1.448138,-1.359270
376,-1.149599,-0.364144,-3.142637,1.705087
403,-1.276590,2.232353,-3.226964,-0.044374
475,0.492936,-0.660194,0.919023,3.127845


**Capping:** Instead of removing outliers, we can cap them at a threshold.

In [263]:
# Set values > 3 to 3, and < -3 to -3, preserving the sign
data[data.abs() > 3] = np.sign(data) * 3

In [264]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.003957,-0.060713,-0.000933,-0.032416
std,0.986860,0.984879,1.018884,0.985750
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.610949,-0.690710,-0.633737,-0.697984
50%,0.002066,-0.058677,-0.015031,-0.062742
75%,0.674379,0.582628,0.677254,0.605943
max,3.000000,3.000000,3.000000,3.000000


**Removal:** Or we can just drop the rows with outliers.

In [265]:
# Keep rows where ALL columns are within the threshold ( < 3)
data[(data.abs() < 3).all(axis="columns")]

,0,1,2,3
0,0.325437,0.044449,-0.494591,1.279838
1,-1.062924,-0.475014,-0.562591,0.041950
2,1.132614,-1.285826,0.356707,-1.237236
3,0.579027,-0.158589,1.369788,-0.673912
4,0.047617,1.619236,1.624633,0.185832
...,...,...,...,...
995,-0.039621,1.649022,-0.255462,0.376047
996,-0.224797,-0.153980,-1.351068,-0.128429
997,1.344547,1.060013,1.249804,0.194880
998,1.367218,1.615782,-0.810448,-1.801196


---

# Section 3: Refinement & Transformation

**Goal:** Now that the data is clean, we need to transform it into the right format for analysis.

We will cover:
1.  **Mapping & Replacing:** Changing values based on a dictionary logic.
2.  **String Manipulation:** Cleaning text data using the `.str` accessor.
3.  **Exporting:** Saving your hard work to a file.

## Part 1: Transforming Data (Mapping)

Sometimes we need to add new columns based on existing ones. `map()` is perfect for this.

In [266]:
data = pd.DataFrame({"food": ["bacon", "pulled pork", "bacon", "pastrami", 
                                "corned beef", "bacon", "pastrami", "honey ham", 
                                "nova lox"],
                         "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,pastrami,6.0
4,corned beef,7.5
5,bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


**Scenario:** We want to add a column indicating the animal source of each food.

In [267]:
meat_to_animal = {
    "bacon": "pig",
    "pulled pork": "pig",
    "pastrami": "cow",
    "corned beef": "cow",
    "honey ham": "pig",
    "nova lox": "salmon"
}

In [268]:
# .map() looks up the value in the 'food' column in our dictionary
data["animal"] = data["food"].map(meat_to_animal)

data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


You can also pass a function to `map()` for custom logic.

In [269]:
def get_animal(x):
    return meat_to_animal[x]

data["food"].map(get_animal)

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

**Replacing Values:** `replace` is a specialized version of map, great for fixing sentinel values (like -999 for missing data).

In [270]:
data = pd.Series([1, -999, 2, -999, -1000, 3])

data

0       1
1    -999
2       2
3    -999
4   -1000
5       3
dtype: int64

In [271]:
# Replace -999 with NaN
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [272]:
# Replace multiple values at once
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [273]:
# Replace with different values (-999 -> NaN, -1000 -> 0)
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [274]:
# Using a dictionary for clarity
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Exercise: Transforming Data

Try replacing values in a DataFrame.

In [275]:
df = pd.DataFrame(np.random.standard_normal((6, 3)))

df.iloc[2:, 1] = -999
df.iloc[4:, 2] = 999

df

,0,1,2
0,0.373952,0.423121,2.483728
1,1.111528,0.615404,-1.057059
2,-0.159944,-999.000000,0.247591
3,-0.456275,-999.000000,0.843533
4,1.005508,-999.000000,999.000000
5,0.519086,-999.000000,999.000000


> **Task:** Replace -999 with NaN and 999 with 0.

In [276]:
df.replace({-999: np.nan, 999: 0}, inplace=True)
df.convert_dtypes

<bound method NDFrame.convert_dtypes of           0         1         2
0  0.373952  0.423121  2.483728
1  1.111528  0.615404 -1.057059
2 -0.159944       NaN  0.247591
3 -0.456275       NaN  0.843533
4  1.005508       NaN  0.000000
5  0.519086       NaN  0.000000>

## Part 2: String Manipulation

Pandas has a special accessor `.str` that unlocks string methods for an entire Series at once. This handles missing values gracefully.

In [277]:
data = {"Dave": "dave@google.com", "Steve": "steve@gmail.com",
        "Rob": "rob@gmail.com", "Wes": np.nan}

data = pd.Series(data)

data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [278]:
# Check if 'gmail' exists in each string
data.str.contains("gmail")

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

Note on Data Types: Pandas has a specialized `StringDType` (`string`) vs the generic `object` type.

In [279]:
data_as_string = data.astype('string')

data_as_string

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                 <NA>
dtype: string

In [280]:
data_as_string.str.contains("gmail")

Dave     False
Steve     True
Rob       True
Wes       <NA>
dtype: boolean

**Slicing:** We can treat the column like a Python string.

In [281]:
# Get the first 5 characters
data_as_string.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes       <NA>
dtype: string

**Regex:** Regular expressions allow for complex pattern matching.

In [282]:
import re

In [283]:
# Pattern to identify email parts: (username) @ (domain) . (suffix)
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"

data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

**Retrieving elements:** We can chain `.str` calls to get specific parts of the regex match.

In [284]:
matches = data.str.findall(pattern, flags=re.IGNORECASE).str[0]

matches

Dave     (dave, google, com)
Steve    (steve, gmail, com)
Rob        (rob, gmail, com)
Wes                      NaN
dtype: object

In [285]:
# Get index 1 of the tuple (the domain name)
matches.str.get(1)

Dave     google
Steve     gmail
Rob       gmail
Wes         NaN
dtype: object

The `extract` method is very powerful—it creates a new DataFrame with columns for each captured regex group.

In [286]:
data.str.extract(pattern, flags=re.IGNORECASE)

,0,1,2
Dave,dave,google,com
Steve,steve,gmail,com
Rob,rob,gmail,com
Wes,NaN,NaN,NaN


### Exercise: Strings

> 1. Get the 1st group of the regex email pattern (the username).
> 2. Convert data into titlecase (e.g. "Dave@Google.Com").

In [287]:
print(matches.str.get(0))
print("\n",data.str.title())

Dave      dave
Steve    steve
Rob        rob
Wes        NaN
dtype: object

 Dave     Dave@Google.Com
Steve    Steve@Gmail.Com
Rob        Rob@Gmail.Com
Wes                  NaN
dtype: object


## Part 3: Writing Data (Exporting)

Once your data is cleaned, you need to save it. `to_csv` is the most common method.

In [288]:
result.to_csv("../data/out.csv")

In [289]:
!cat ../data/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,


**Tip:** You usually want `index=False` to avoid saving the row numbers as a separate column.

In [290]:
result.to_csv("../data/out.csv", index=False)

In [291]:
!cat ../data/out.csv

something,a,b,c,d,message
one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,


**Excel Export:**

In [292]:
# Option 1: Using ExcelWriter (good for multiple sheets)
writer = pd.ExcelWriter('../data/out.xlsx')

frame.to_excel(writer, 'Sheet1')

writer.close()

In [293]:
# Option 2: Direct export
frame.to_excel('../data/out.xlsx')

---

## Optional Topics

If time permits, explore these advanced transformations.

### Renaming Axis Indexes
Changing row/column labels using mapping.

In [294]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])

In [295]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


Using `.map()` on the index:

In [296]:
def transform(x):
    return x[:4].upper()

data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [297]:
# Assign back to modify in-place
data.index = data.index.map(transform)

In [298]:
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


Using `.rename()` (returns a copy by default):

In [299]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [300]:
data.rename(index={"OHIO": "INDIANA"}, columns={"three": 3})

,one,two,3,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### Permutation and Random Sampling
Reordering or selecting random subsets.

In [357]:
df = pd.DataFrame(np.arange(5 * 7).reshape((5, 7)))

df

,0,1,2,3,4,5,6
0,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20
3,21,22,23,24,25,26,27
4,28,29,30,31,32,33,34


In [358]:
df.shape

(5, 7)

In [359]:
# Create a random order of indices
sampler = np.random.permutation(5)

sampler

array([4, 3, 1, 2, 0])

In [360]:
# Reorder rows
df.iloc[sampler]

,0,1,2,3,4,5,6
4,28,29,30,31,32,33,34
3,21,22,23,24,25,26,27
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20
0,0,1,2,3,4,5,6


In [361]:
# Alternative: use .take()
df.take(sampler)

,0,1,2,3,4,5,6
4,28,29,30,31,32,33,34
3,21,22,23,24,25,26,27
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20
0,0,1,2,3,4,5,6


In [363]:
df.shape

(5, 7)

Permuting columns:

In [364]:
column_sampler = np.random.permutation(df.shape[1])

column_sampler

array([0, 4, 3, 6, 1, 2, 5])

In [306]:
df.take(column_sampler, axis=1)

,6,5,4,3,0,2,1
0,6,5,4,3,0,2,1
1,13,12,11,10,7,9,8
2,20,19,18,17,14,16,15
3,27,26,25,24,21,23,22
4,34,33,32,31,28,30,29


**Random Sample:** Getting a random subset without manually creating a sampler.

In [367]:
df.sample(n=3)

,0,1,2,3,4,5,6
2,14,15,16,17,18,19,20
1,7,8,9,10,11,12,13
4,28,29,30,31,32,33,34


In [308]:
choices = pd.Series([5, 7, -1, 6, 4])

# Sample with replacement (items can be picked more than once)
choices.sample(n=10, replace=True)

4    4
2   -1
0    5
3    6
3    6
1    7
4    4
3    6
2   -1
1    7
dtype: int64

> **Task:** Sample `df` using the parameter `frac` (percentage) instead of `n` (count).

### Categorical Data
Converting string columns to `category` dtype saves memory and speeds up operations.

> **📊 Visual Illustration Available**: See ![categorical_binning](../visualisations/05_categorical_binning.png` for visualizations comparing `pd.cut()` vs `pd.qcut()`, categorical operations, and dummy variables.


In [309]:
values = pd.Series(["apple", "orange", "apple", "apple"] * 2)

values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [310]:
values.unique()

array(['apple', 'orange'], dtype=object)

In [311]:
values.value_counts()

apple     6
orange    2
dtype: int64

**Under the hood:** Categoricals are stored as integers referencing a dictionary of values.

In [368]:
values = pd.Series([0, 1, 0, 0] * 2)

dim = pd.Series(["apple", "orange"])

values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [313]:
dim

0     apple
1    orange
dtype: object

In [314]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

**Using Pandas `category` type:**

In [315]:
fruits = ["apple", "orange", "apple", "apple"] * 2
N = len(fruits)

# to ensure reproducibility
rng = np.random.default_rng(seed=12345)

df = pd.DataFrame({"fruit": fruits, 
                   "basket_id": np.arange(N), 
                   "count": rng.integers(3, 15, size=N), 
                   "weight": rng.uniform(0, 4, size=N)}, 
                  columns=["basket_id", "fruit", "count", "weight"])

df

,basket_id,fruit,count,weight
0,0,apple,11,1.564438
1,1,orange,5,1.331256
2,2,apple,12,2.393235
3,3,apple,6,0.746937
4,4,apple,5,2.691024
5,5,orange,12,3.767211
6,6,apple,10,0.992983
7,7,apple,11,3.795525


In [316]:
# Convert object column to category
fruit_cat = df['fruit'].astype('category')

fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

The values for `fruit_cat` are now an instance of `pandas.Categorical`, which you can access via the `.array` attribute:

In [317]:
c = fruit_cat.array

type(c)

pandas.core.arrays.categorical.Categorical

In [318]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [319]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [320]:
dict(enumerate(c.categories))

{0: 'apple', 1: 'orange'}

In [321]:
# Assign back to the dataframe
df['fruit'] = df['fruit'].astype('category')

df

,basket_id,fruit,count,weight
0,0,apple,11,1.564438
1,1,orange,5,1.331256
2,2,apple,12,2.393235
3,3,apple,6,0.746937
4,4,apple,5,2.691024
5,5,orange,12,3.767211
6,6,apple,10,0.992983
7,7,apple,11,3.795525


**Creating from Codes:**

In [322]:
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]

my_cats = pd.Categorical.from_codes(codes, categories)

my_cats

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

**Ordered Categoricals:** Useful for Likert scales or sizes (Small < Medium < Large).

In [323]:
ordered_cats = pd.Categorical.from_codes(codes, categories, ordered=True)

ordered_cats

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

**Binning Data (`pd.cut`):** Converting continuous data (e.g., Age) into categorical bins (e.g., Age Groups).

In [324]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [325]:
# Define bin edges
bins = [18, 25, 35, 60, 100]

# Cut the data
age_cat = pd.cut(ages, bins)

age_cat

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [326]:
age_cat.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [327]:
age_cat.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [328]:
age_cat.categories[0]

Interval(18, 25, closed='right')

In [329]:
pd.value_counts(age_cat)

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

**Binning Parameters:**

In [330]:
# right=False makes the left side closed [inclusive, exclusive)
pd.cut(ages, bins, right=False)

[[18, 25), [18, 25), [25, 35), [25, 35), [18, 25), ..., [25, 35), [60, 100), [35, 60), [35, 60), [25, 35)]
Length: 12
Categories (4, interval[int64, left]): [[18, 25) < [25, 35) < [35, 60) < [60, 100)]

In [331]:
# Adding custom labels
group_names = ["Youth", "YoungAdult", "MiddleAged", "Senior"]

pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

If you pass an integer number of bins to `pd.cut`, it will compute equal-length bins based on the min/max of the data.

In [332]:
data = np.random.uniform(size=20)

pd.cut(data, 4, precision=2)

[(0.77, 1.0], (0.55, 0.77], (0.098, 0.32], (0.77, 1.0], (0.098, 0.32], ..., (0.55, 0.77], (0.32, 0.55], (0.77, 1.0], (0.32, 0.55], (0.32, 0.55]]
Length: 20
Categories (4, interval[float64, right]): [(0.098, 0.32] < (0.32, 0.55] < (0.55, 0.77] < (0.77, 1.0]]

> **Task:** Cut `ages` into 5 bins instead of 4. Set the labels to `['Youth', 'YoungAdult', 'MiddleAged', 'Senior', 'Elderly']`.

**Categorical Methods (.cat):**

In [333]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')

cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [334]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [335]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

Modifying categories:

In [336]:
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)

cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): ['a', 'b', 'c', 'd', 'e']

In [337]:
cat_s.value_counts()

a    2
b    2
c    2
d    2
dtype: int64

In [338]:
cat_s2.value_counts()

a    2
b    2
c    2
d    2
e    0
dtype: int64

In [339]:
cat_s2.cat.remove_unused_categories()

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

**Computing Indicator / Dummy Variables:** Converting categorical variables into binary columns (One-Hot Encoding) for machine learning.

In [340]:
df = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"], 
                   "data1": range(6)})

df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [341]:
pd.get_dummies(df["key"])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [342]:
dummies = pd.get_dummies(df["key"], prefix="key")

dummies

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


Recipe: Combining `get_dummies` with `cut`.

In [343]:
np.random.seed(12345)

values = np.random.uniform(size=10)

values


array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [344]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


> **Task:** Cut values into 4 bins and create dummy variables.

### Databases

Connecting to a database using `sqlalchemy`.

In [345]:
import sqlalchemy as sqla

In [346]:
import os 

parent_dir = os.path.abspath(os.path.pardir)

In [347]:
# Create connection string
engine = sqla.create_engine(f'duckdb:///{parent_dir}/data/unit-1-4.db')

In [348]:
# Read entire table
df = pd.read_sql('resale_flat_prices_2017', engine)

/home/khoonseng/miniconda3/envs/pds/lib/python3.10/site-packages/duckdb_engine/__init__.py:184: DuckDBEngineWarning: duckdb-engine doesn't yet support reflection on indices
  warnings.warn(


In [349]:
df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0
...,...,...,...,...,...,...,...,...,...,...,...
198037,2025-01,YISHUN,5 ROOM,265,YISHUN ST 22,01 TO 03,122.0,Improved,1986,60 years 11 months,625000.0
198038,2025-01,YISHUN,5 ROOM,511B,YISHUN ST 51,10 TO 12,113.0,Improved,2017,91 years 09 months,730000.0
198039,2025-01,YISHUN,5 ROOM,871,YISHUN ST 81,07 TO 09,122.0,Improved,1988,62 years 04 months,738000.0
198040,2025-01,YISHUN,5 ROOM,851,YISHUN ST 81,10 TO 12,127.0,Improved,1988,62 years 04 months,740000.0


In [350]:
# Execute raw SQL query
df = pd.read_sql("SELECT * FROM resale_flat_prices_2017", engine)

In [351]:
engine.table_names()

/tmp/ipykernel_1253/4162273999.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['resale_flat_prices_2017']

### Binary files (Pickle)

`pickle` is Python's native serialization format. Good for short-term storage, but not for sharing data between different languages.

In [352]:
result.to_pickle('../data/out.pkl')

In [353]:
data = pd.read_pickle('../data/out.pkl')

data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


> **Task:** Write a filtered DataFrame to a new database table.
> 1. Filter for flats in "YISHUN".
> 2. Write to table `yishun_flat_prices_2017`.

In [354]:
df_yishun = df[df.town == "YISHUN"]

df_yishun.to_sql("yishun_flat_prices_2017", engine)

-1

> **Final Challenge:** 
> 1. Read only flats from `BISHAN` to a new dataframe.
> 2. Write the dataframe to a new database table `bishan_flat_prices_2017`.

In [1]:
import pandas as pd

df = pd.DataFrame({'A': [1, 2, 3, 4, 5], 'B': [10, 20, 30, 40, 50]})
threshold = 30


In [5]:
#df[(df > threshold)].dropna(how="all")
#df[(df > threshold)].dropna(thresh=1)
df[(df > threshold).any(axis=1)]

,A,B
3,4,40
4,5,50


In [386]:
import pandas as pd

df = pd.DataFrame({'A': [2, 1, 2], 'B': [2, 3, 1], 'C': [1, 2, 3]})
df

,A,B,C
0,2,2,1
1,1,3,2
2,2,1,3


In [391]:
df.sort_values(by=['A','B'], ascending=[True,True], inplace=True)
df

,A,B,C
1,1,3,2
2,2,1,3
0,2,2,1


In [8]:
import pandas as pd

df1 = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
df2 = pd.DataFrame({'A': [7, 8, 9], 'B': [10, 11, 12]})
print(df1)
print(df2)

   A  B
0  1  4
1  2  5
2  3  6
   A   B
0  7  10
1  8  11
2  9  12


In [10]:
print(pd.concat([df1, df2], axis=0, ignore_index=True))

   A   B
0  1   4
1  2   5
2  3   6
3  7  10
4  8  11
5  9  12


In [409]:
import pandas as pd

df = pd.DataFrame({'A': [1, 2, 3, 4, 5]})

In [418]:
print(df['A'].cumsum(axis=0))


0     1
1     3
2     6
3    10
4    15
Name: A, dtype: int64


In [419]:
import pandas as pd

series = pd.Series(['apple', 'banana', 'cherry'])

In [420]:
print(series.str.upper())

0     APPLE
1    BANANA
2    CHERRY
dtype: object
